In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [190]:
# get broadband data
broadband_data = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'data\\merged_broadband.csv'))
zipcode = broadband_data.Zip.unique()
broadband_data.head()
# zipcode

,Unnamed: 0,Zip,Population,WiredCount_2020,Fwcount_2020,AllProviderCount_2020,Wired25_3_2020,Wired100_3_2020,All25_3_2020,All100_3,...,All25_3_2015,All100_3.1,Total_Enrolled_Households,ST,COUNTY NAME,COUNTY ID,BROADBAND USAGE,ERROR RANGE (MAE)(+/-),ERROR RANGE (95%)(+/-),MSD
0,0,29639,1742.0,3.0,0.0,8.0,3.0,3.0,5.0,3.0,...,3.0,3.0,21.0,SC,Abbeville,45001,0.948,0.034,0.110,0.002
1,1,29620,12934.0,6.0,0.0,11.0,5.0,3.0,7.0,3.0,...,3.0,3.0,542.0,SC,Abbeville,45001,0.398,0.002,0.007,0.000
2,2,29659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,SC,Abbeville,45001,0.206,0.152,0.608,0.043
3,3,29638,2944.0,6.0,1.0,13.0,4.0,4.0,6.0,4.0,...,2.0,2.0,68.0,SC,Abbeville,45001,0.369,0.010,0.031,-0.001
4,4,29628,2759.0,4.0,0.0,8.0,3.0,2.0,5.0,2.0,...,3.0,3.0,102.0,SC,Abbeville,45001,0.221,0.014,0.043,0.000


In [197]:
print(f'Total number of unique zipcode in broadband data: {len(zipcode)}')

Total number of unique zipcode in broadband data: 32653


In [187]:
#get census data
census_data = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'data\\relabeled_census.csv'))
census_data.head()


,NAME,median_age_overall,median_age_male,median_age_female,state,county,tract,employment_rate,median_income,total_households,...,pct_internet_broadband_satellite,pct_internet_only_satellite,pct_internet_other,pct_internet_no_subscrp,pct_internet_none,pct_computer,pct_computer_with_dialup,pct_computer_with_broadband,pct_computer_no_internet,pct_no_computer
0,"Census Tract 11, Jefferson County, Alabama",39.0,42.5,38.1,1,73,1100,51.0,37030,1851,...,0.090222,0.009184,0.0,0.011345,0.248514,0.808212,0.0,0.740140,0.068071,0.191788
1,"Census Tract 14, Jefferson County, Alabama",44.3,40.5,49.1,1,73,1400,45.4,36066,816,...,0.049020,0.000000,0.0,0.020833,0.254902,0.856618,0.0,0.710784,0.145833,0.143382
2,"Census Tract 20, Jefferson County, Alabama",34.0,31.0,36.4,1,73,2000,47.7,27159,1419,...,0.046512,0.000000,0.0,0.000000,0.454545,0.713178,0.0,0.545455,0.167724,0.286822
3,"Census Tract 38.02, Jefferson County, Alabama",35.8,31.7,37.3,1,73,3802,51.7,38721,1894,...,0.039599,0.000000,0.0,0.063358,0.336325,0.857445,0.0,0.594509,0.262936,0.142555
4,"Census Tract 40, Jefferson County, Alabama",52.1,51.6,53.8,1,73,4000,36.9,18525,1429,...,0.045486,0.019594,0.0,0.051085,0.475157,0.630511,0.0,0.447866,0.182645,0.369489


In [188]:
print(f'Total number of rows in census data: {len(census_data.index)}')

Total number of rows in census data: 73056


In [191]:
# get the mapping between state name and USPS
state_data = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'data\\zip_conversation_data\\state_name_conversion.csv'))
state_data_dict = state_data.set_index('Full Name').to_dict()['USPS']
# state_data_dict

In [194]:
#get zipcode, county name, and state for mapping
tract_data = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'data\\\zip_conversation_data\\all_states_zip_conversion.csv'))
tract_data = tract_data.drop(0)
tract_data = tract_data.reset_index()
state =tract_data.cntyname.str.split(' ').str[-1].to_frame()
state.columns = ['state_name']
county = tract_data.cntyname.str.split(' ').str[:-1].str.join(' ').to_frame()
county.columns = ['county_name']
county = county.join(state)
tract_data = tract_data.join(county)
tract_data = tract_data[['zcta5',  'county_name', 'state_name']]

tract_data.head()

,zcta5,county_name,state_name
0,01001,Hampden,MA
1,01001,Hampden,MA
2,01001,Hampden,MA
3,01001,Hampden,MA
4,01001,Hampden,MA


In [195]:
print(f'Total number of unique zipcode in mapping table: {len(tract_data.zcta5.unique())}')


Total number of unique zipcode in mapping table: 32847


In [196]:
#select zipcode covered in broadband data
tract_data = tract_data[tract_data.zcta5.isin(zipcode)]
tract_data.head()
print(f'Total number of unique zipcode in both mapping table and broadband data: {len(tract_data.zcta5.unique())}')

Total number of unique zipcode in both mapping table and broadband data: 18309


In [198]:
#convert state/county code to state/county name for mapping
census_tract = pd.DataFrame(census_data.NAME.str.split(',').tolist(), columns = ['tract', 'county', 'state'])
census_tract.tract = census_tract.tract.str.split(' ').str[-1]
census_tract.state = census_tract.state.str.strip()
census_tract = census_tract.replace({'state':state_data_dict})
census_tract = census_tract[['county', 'state']]
census_tract.county = census_tract.county.str.replace('County','').str.strip()
census_data[['county', 'state']] = census_tract[['county', 'state']]
census_data.head()
# len(census_data.index)

,NAME,median_age_overall,median_age_male,median_age_female,state,county,tract,employment_rate,median_income,total_households,...,pct_internet_broadband_satellite,pct_internet_only_satellite,pct_internet_other,pct_internet_no_subscrp,pct_internet_none,pct_computer,pct_computer_with_dialup,pct_computer_with_broadband,pct_computer_no_internet,pct_no_computer
0,"Census Tract 11, Jefferson County, Alabama",39.0,42.5,38.1,AL,Jefferson,1100,51.0,37030,1851,...,0.090222,0.009184,0.0,0.011345,0.248514,0.808212,0.0,0.740140,0.068071,0.191788
1,"Census Tract 14, Jefferson County, Alabama",44.3,40.5,49.1,AL,Jefferson,1400,45.4,36066,816,...,0.049020,0.000000,0.0,0.020833,0.254902,0.856618,0.0,0.710784,0.145833,0.143382
2,"Census Tract 20, Jefferson County, Alabama",34.0,31.0,36.4,AL,Jefferson,2000,47.7,27159,1419,...,0.046512,0.000000,0.0,0.000000,0.454545,0.713178,0.0,0.545455,0.167724,0.286822
3,"Census Tract 38.02, Jefferson County, Alabama",35.8,31.7,37.3,AL,Jefferson,3802,51.7,38721,1894,...,0.039599,0.000000,0.0,0.063358,0.336325,0.857445,0.0,0.594509,0.262936,0.142555
4,"Census Tract 40, Jefferson County, Alabama",52.1,51.6,53.8,AL,Jefferson,4000,36.9,18525,1429,...,0.045486,0.019594,0.0,0.051085,0.475157,0.630511,0.0,0.447866,0.182645,0.369489


In [199]:
print(f'Total number of rows in census data: {len(census_data.index)}')

Total number of rows in census data: 73056


In [200]:
#remove duplicated entries in zip code
tract_data = tract_data.drop_duplicates(subset='zcta5', keep='first')
print(f'Total number of unique zipcode in  mapping table : {len(tract_data.zcta5.unique())}')

Total number of unique zipcode in  mapping table : 18309


In [206]:
#merge census data with zipcode on county_name and state_name
res = census_data.merge(tract_data, how = 'left', right_on=['county_name', 'state_name'], left_on=['county', 'state'])
res= res.drop_duplicates(subset='zcta5', keep='first')
res = res[~res.zcta5.isnull()]
res.rename(columns = {'zcta5':'zipcode'}, inplace = True)
res.head()

,NAME,median_age_overall,median_age_male,median_age_female,state,county,tract,employment_rate,median_income,total_households,...,pct_internet_no_subscrp,pct_internet_none,pct_computer,pct_computer_with_dialup,pct_computer_with_broadband,pct_computer_no_internet,pct_no_computer,zipcode,county_name,state_name
1181,"Census Tract 2, Aleutians West Census Area, Al...",-1.0,-1.0,-1.0,AK,Aleutians West Census Area,200,-1.0,-1,-1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,99546,Aleutians West Census Area,AK
1182,"Census Tract 2, Aleutians West Census Area, Al...",-1.0,-1.0,-1.0,AK,Aleutians West Census Area,200,-1.0,-1,-1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,99547,Aleutians West Census Area,AK
1183,"Census Tract 2, Aleutians West Census Area, Al...",-1.0,-1.0,-1.0,AK,Aleutians West Census Area,200,-1.0,-1,-1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,99591,Aleutians West Census Area,AK
1184,"Census Tract 2, Aleutians West Census Area, Al...",-1.0,-1.0,-1.0,AK,Aleutians West Census Area,200,-1.0,-1,-1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,99638,Aleutians West Census Area,AK
1185,"Census Tract 2, Aleutians West Census Area, Al...",-1.0,-1.0,-1.0,AK,Aleutians West Census Area,200,-1.0,-1,-1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,99660,Aleutians West Census Area,AK


In [207]:
print(f'Total number of unique zipcode after merge : {len(res.zipcode.unique())}')

Total number of unique zipcode after merge : 18162


In [211]:
#merge census data with broadband data
merged_census_broadband_data = res.merge(broadband_data, how = 'inner', left_on=['zipcode'], right_on=['Zip'])
merged_census_broadband_data.head()

,NAME,median_age_overall,median_age_male,median_age_female,state,county,tract,employment_rate,median_income,total_households,...,All25_3_2015,All100_3.1,Total_Enrolled_Households,ST,COUNTY NAME,COUNTY ID,BROADBAND USAGE,ERROR RANGE (MAE)(+/-),ERROR RANGE (95%)(+/-),MSD
0,"Census Tract 2, Aleutians West Census Area, Al...",-1.0,-1.0,-1.0,AK,Aleutians West Census Area,200,-1.0,-1,-1,...,0.0,0.0,1.0,AK,Aleutians West Census Area,2016,0.043,0.152,0.608,0.043
1,"Census Tract 2, Aleutians West Census Area, Al...",-1.0,-1.0,-1.0,AK,Aleutians West Census Area,200,-1.0,-1,-1,...,0.0,0.0,NaN,AK,Aleutians West Census Area,2016,0.000,0.197,0.869,0.022
2,"Census Tract 2, Aleutians West Census Area, Al...",-1.0,-1.0,-1.0,AK,Aleutians West Census Area,200,-1.0,-1,-1,...,0.0,0.0,NaN,AK,Aleutians West Census Area,2016,0.000,0.197,0.869,0.022
3,"Census Tract 2, Aleutians West Census Area, Al...",-1.0,-1.0,-1.0,AK,Aleutians West Census Area,200,-1.0,-1,-1,...,0.0,0.0,NaN,AK,Aleutians West Census Area,2016,0.000,0.197,0.869,0.022
4,"Census Tract 2, Aleutians West Census Area, Al...",-1.0,-1.0,-1.0,AK,Aleutians West Census Area,200,-1.0,-1,-1,...,0.0,0.0,NaN,AK,Aleutians West Census Area,2016,0.000,0.065,0.228,0.013


In [212]:
print(f'Total number of unique zipcode after merge : {len(merged_census_broadband_data.zipcode.unique())}')

Total number of unique zipcode after merge : 18162


In [216]:
check_na = merged_census_broadband_data.isnull().sum()

In [219]:
check_na[check_na>0]/18162

poverty_rate                                0.015142
pct_pop_bachelors+                          0.011893
pct_pop_hs+                                 0.011893
pct_internet                                0.013600
pct_internet_dial_up                        0.013600
pct_internet_broadband_any_type             0.013600
pct_internet_cellular                       0.013600
pct_only_cellular                           0.013600
pct_internet_broadband_fiber                0.013600
pct_internet_broadband_satellite            0.013600
pct_internet_only_satellite                 0.013600
pct_internet_other                          0.013600
pct_internet_no_subscrp                     0.013600
pct_internet_none                           0.013600
pct_computer                                0.013600
pct_computer_with_dialup                    0.013600
pct_computer_with_broadband                 0.013600
pct_computer_no_internet                    0.013600
pct_no_computer                             0.